# Welcome to Lab Session 3 - Derek Thomas - 07Oct2017

## Define the environment for the Dataframe

In [3]:
import pandas as pd
import numpy as np
#import pandas_datareader as pdr
#from pandas_datareader import data, wb
import urllib3
http = urllib3.PoolManager()
import string
import re
import json
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

## Import the Data Breaches Dataframe and parse relevant columns

In [4]:
df = pd.read_excel('Information is Beautiful_ Data Breaches (public).xlsx', header=1, sheetname='2017 update',parse_cols=('A,D:G,J'))

In [5]:
df.head(4)

,Entity,YEAR,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
0,NaN,"years are encoded (0=2004, 8 = 2012, 9 = 2013,...","(use 3m, 4m, 5m or 10m to approximate unknown ...",NaN,NaN,1. Just email address/Online information 20 SS...
1,AOL,0,92000000,web,inside job,1
2,Automatic Data Processing,1,125000,financial,poor security,20
3,Ameritrade Inc.,1,200000,financial,lost / stolen device,20


## Modify column names

In [6]:
df.columns=['Entity',\
            'Year',\
            'Records Lost',\
            'Organization',\
            'Method of Leak',\
            'Sensitivity']
df.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
0,NaN,"years are encoded (0=2004, 8 = 2012, 9 = 2013,...","(use 3m, 4m, 5m or 10m to approximate unknown ...",NaN,NaN,1. Just email address/Online information 20 SS...
1,AOL,0,92000000,web,inside job,1
2,Automatic Data Processing,1,125000,financial,poor security,20
3,Ameritrade Inc.,1,200000,financial,lost / stolen device,20


## Delete irrelevant narrartive row

In [7]:
df = df.drop(df.index[0])
df.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
1,AOL,0,92000000,web,inside job,1
2,Automatic Data Processing,1,125000,financial,poor security,20
3,Ameritrade Inc.,1,200000,financial,lost / stolen device,20
4,Citigroup,1,3900000,financial,lost / stolen device,300


## Convert Year index to Calendar Year value

In [8]:
def ConvertYear(x):
    if (x != '14'):
        return str((int(x) + 2004))
    else:
        return ('2017')
df['Year'] = df.Year.apply(ConvertYear)
df.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
1,AOL,2004,92000000,web,inside job,1
2,Automatic Data Processing,2005,125000,financial,poor security,20
3,Ameritrade Inc.,2005,200000,financial,lost / stolen device,20
4,Citigroup,2005,3900000,financial,lost / stolen device,300


## Convert all "Organization" descriptions to Captialized case

In [9]:
df['Organization'] = df['Organization'].str.capitalize()
df.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
1,AOL,2004,92000000,Web,inside job,1
2,Automatic Data Processing,2005,125000,Financial,poor security,20
3,Ameritrade Inc.,2005,200000,Financial,lost / stolen device,20
4,Citigroup,2005,3900000,Financial,lost / stolen device,300


##  Convert all "Methods of Leak" descriptions to Captialized case

In [10]:
df['Method of Leak'] = df['Method of Leak'].str.capitalize()
df.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
1,AOL,2004,92000000,Web,Inside job,1
2,Automatic Data Processing,2005,125000,Financial,Poor security,20
3,Ameritrade Inc.,2005,200000,Financial,Lost / stolen device,20
4,Citigroup,2005,3900000,Financial,Lost / stolen device,300


## Update Records Lost with unknown values to 5m (5 million)

In [11]:
df[df['Records Lost'].isnull()]

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity
147,OVH,2013,NaN,Web,Hacked,20
163,UbiSoft,2013,NaN,Gaming,Hacked,20


In [12]:
df['Records Lost'][df['Records Lost'].isnull()] = 5000000   # Replace any "null" Records Lost values with 5 million (5000000)
df[df['Records Lost'].isnull()]                             # Verify there are no remaining "null" Records Lost values

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity


## Ensure all Sensitivity Codes are of numeric data type

In [13]:
df['Sensitivity'] = df['Sensitivity'].apply(pd.to_numeric)

## Update invalid/undefined Sensitivity Code records with valid Code of:  1

In [14]:

df.Sensitivity[~df['Sensitivity'].isin([1,20,300,4000,50000])] = 1

## Verify all Sensitivity Codes are now valid

In [15]:
df[~df['Sensitivity'].isin([1,20,300,4000,50000])]

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity


In [16]:
df.iloc[133]

Entity            South Carolina State Dept. of Revenue
Year                                               2012
Records Lost                                    3600000
Organization                                 Government
Method of Leak                                   Hacked
Sensitivity                                           1
Name: 134, dtype: object

## Add new column containing text string corresponding to Sensitivity Code
1. Just email address/Online information 20 SSN/Personal details 300 Credit card information 4000 Email password/Health records 50000 Full bank account details

In [17]:
def getSensitivityText(x):
    if (x == 1):
        return 'Email Address/Online Information'
    elif (x == 20):
        return 'SSN/Personal Details'
    elif (x == 300):
        return 'Credit Card Information'
    elif (x == 4000):
        return 'Email Password/Health Records'
    elif (x == 50000):
        return 'Full Bank Account Details'
    else:
        return 'Other'
df2 = df.copy()
df2['Sensitivity of Information'] = df.Sensitivity.apply(getSensitivityText)
df2.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity,Sensitivity of Information
1,AOL,2004,92000000,Web,Inside job,1,Email Address/Online Information
2,Automatic Data Processing,2005,125000,Financial,Poor security,20,SSN/Personal Details
3,Ameritrade Inc.,2005,200000,Financial,Lost / stolen device,20,SSN/Personal Details
4,Citigroup,2005,3900000,Financial,Lost / stolen device,300,Credit Card Information


## Import the Stock Ticker lookup file

## Remove miscellaneous punctuation marks and format values for URL search

In [18]:
urlString = df2['Entity']

In [19]:
def formatUrlString(x):
    punctuations = '''!()-[]{};:'"''\,<>./?@#$%^&*_~'''
    cleanEntity = ''
    for singleChar in x:
        singleChar = singleChar.replace(u'\u2019', '')        # Additional filtering for unwanted Unicode apostrophes
        if singleChar not in punctuations:
            cleanEntity = (cleanEntity + singleChar)
    return cleanEntity.strip().replace(' ', '%20')            # Format URL Query String by inserting code for blank spaces
urlString = urlString.apply(formatUrlString)
urlString.head(4)

1                              AOL
2    Automatic%20Data%20Processing
3                 Ameritrade%20Inc
4                        Citigroup
Name: Entity, dtype: object

In [20]:
df2['Date of Breach'] = np.nan

In [21]:
df2.head(4)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity,Sensitivity of Information,Date of Breach
1,AOL,2004,92000000,Web,Inside job,1,Email Address/Online Information,NaN
2,Automatic Data Processing,2005,125000,Financial,Poor security,20,SSN/Personal Details,NaN
3,Ameritrade Inc.,2005,200000,Financial,Lost / stolen device,20,SSN/Personal Details,NaN
4,Citigroup,2005,3900000,Financial,Lost / stolen device,300,Credit Card Information,NaN


## Obtain top 20 Entities with most Records Lost in a Data Breach 

In [24]:
df2.sort_values(by = 'Records Lost', ascending = False).head(20)

,Entity,Year,Records Lost,Organization,Method of Leak,Sensitivity,Sensitivity of Information,Date of Breach
246,River City Media,2017,1370000000,Web,Accidentally published,20,SSN/Personal Details,NaN
164,Yahoo,2013,1000000000,Web,Hacked,20,SSN/Personal Details,NaN
253,Spambot,2018,711000000,Web,Poor security,4000,Email Password/Health Records,NaN
187,Yahoo,2014,500000000,Web,Hacked,20,SSN/Personal Details,NaN
245,Friend Finder Network,2017,412000000,Web,Hacked,1,Email Address/Online Information,NaN
132,Court Ventures,2012,200000000,Financial,Inside job,20,SSN/Personal Details,NaN
209,Deep Root Analytics,2015,198000000,Web,Poor security,20,SSN/Personal Details,NaN
221,MySpace,2016,164000000,Web,Hacked,1,Email Address/Online Information,NaN
131,Massive American business hack,2012,160000000,Financial,Hacked,50000,Full Bank Account Details,NaN
186,Ebay,2014,145000000,Web,Hacked,1,Email Address/Online Information,NaN


In [25]:
dfStocks = pd.DataFrame(df2['Entity'])
dfStocks['URL String'] = urlString
dfStocks['Tickers'] = np.nan
dfStocks.head(4)

,Entity,URL String,Tickers
1,AOL,AOL,NaN
2,Automatic Data Processing,Automatic%20Data%20Processing,NaN
3,Ameritrade Inc.,Ameritrade%20Inc,NaN
4,Citigroup,Citigroup,NaN


## Write Dataframe to ".csv" file for import into Tableau

In [26]:
df2.to_csv('DataBreachLab3.csv')